In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/05/07 16:52:29 WARN Utils: Your hostname, arkadia-355.local resolves to a loopback address: 127.0.0.1; using 10.0.1.12 instead (on interface en0)
23/05/07 16:52:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 16:52:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/07 16:52:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


23/05/07 16:52:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
from pyspark.sql import types

In [5]:
schema = types.StructType([
    types.StructField('type', types.StringType(), True), 
    types.StructField('key', types.StringType(), True), 
    types.StructField('revision', types.IntegerType(), True), 
    types.StructField('last_modified', types.TimestampType(), True), 
    types.StructField('json', types.StringType(), True)
])

In [6]:
df = (
    spark.read
    .option("sep", "\t")
    .option("header", "false")
    .schema(schema)
    .csv("ol_dump_authors_latest.txt")
)

In [7]:
df = df.repartition(24)

In [18]:
df.write.parquet("ol/authors/latest", mode="overwrite")

23/05/07 16:26:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/05/07 16:26:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/05/07 16:26:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/05/07 16:26:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/05/07 16:26:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/05/07 16:26:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/05/07 16:26:07 WARN MemoryManager: Total allocation exceeds 95.

In [8]:
df = spark.read.parquet("ol/authors/latest")

In [9]:
df.printSchema()

root
 |-- type: string (nullable = true)
 |-- key: string (nullable = true)
 |-- revision: integer (nullable = true)
 |-- last_modified: string (nullable = true)
 |-- json: string (nullable = true)



In [ ]:
df

DataFrame[type: string, key: string, revision: int, last_modified: string, json: string]

In [10]:
df.show()

+------------+--------------------+--------+--------------------+--------------------+
|        type|                 key|revision|       last_modified|                json|
+------------+--------------------+--------+--------------------+--------------------+
|/type/author| /authors/OL6587319A|       1|2009-02-18T11:37:...|{"name": "Dale Sh...|
|/type/author| /authors/OL9810305A|       1|2021-10-10T00:11:...|{"type": {"key": ...|
|/type/author| /authors/OL3877417A|       1|2008-04-30T20:50:...|{"name": "Don Mag...|
|/type/author| /authors/OL8360454A|       1|2020-08-31T21:33:...|{"name": "Floss D...|
|/type/author| /authors/OL2899506A|       1|2008-04-29T15:03:...|{"name": "Girma B...|
|/type/author| /authors/OL3314602A|       1|2008-04-30T09:38:...|{"name": "Phil Mu...|
|/type/author| /authors/OL9252049A|       1|2021-06-01T08:10:...|{"type": {"key": ...|
|/type/author| /authors/OL3947121A|       1|2008-04-30T20:50:...|{"name": "Anthea ...|
|/type/author|/authors/OL12041121A|       1

In [107]:
from pyspark.sql.functions import col, explode, from_json, explode_outer

In [101]:
schema = types.StructType(
    [
        types.StructField(
            "type", types.MapType(types.StringType(), types.StringType(), True), True
        ),
        types.StructField("name", types.StringType(), True),
        types.StructField("key", types.StringType(), True),
        types.StructField("source_records", types.StringType(), True),
        types.StructField("latest_revision", types.IntegerType(), True),
        types.StructField("revision", types.IntegerType(), True),
        types.StructField(
            "created",
            types.StructType(
                [
                    types.StructField("type", types.StringType(), True),
                    types.StructField("value", types.TimestampType(), True),
                ]
            ),
        ),
        types.StructField("last_modified", types.StringType(), True),
        types.StructField("personal_name", types.StringType(), True),
        types.StructField("birth_date", types.StringType(), True),
    ]
)

https://stackoverflow.com/questions/51070251/pyspark-explode-json-in-column-to-multiple-columns

In [108]:
df.withColumn("json", from_json("json", schema)).select(
    "type",
    "key",
    "revision",
    col("json.created.value").alias("created_at"),
    "last_modified",
    "json.name",
    "json.source_records",
    "json.latest_revision",
    "json.personal_name"
).show()

+------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|        type|                 key|revision|          created_at|       last_modified|                name|      source_records|latest_revision|       personal_name|
+------------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|/type/author| /authors/OL6587319A|       1|2009-02-18 11:37:...|2009-02-18T11:37:...|        Dale Shermon|                null|           null|        Dale Shermon|
|/type/author| /authors/OL9810305A|       1|2021-10-10 00:11:...|2021-10-10T00:11:...|        Sadie Hobbes|["bwb:97986988335...|              1|                null|
|/type/author| /authors/OL3877417A|       1|                null|2008-04-30T20:50:...|          Don Magnus|                null|           null|                null|
|/ty